In [80]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import cm
from matplotlib import animation
import matplotlib.colors as colors
from IPython.display import HTML

%matplotlib inline
cmap = cm.spring

The following functions are used to compute the position, velocity, and acceleration of the moving particle at a time $t$.

In [122]:
w = 1
v0 = 0.1
x0 = v0/w

def path(t):
    return x0*np.sin(w*t), 0, v0*t

def path_deriv(t):
    return x0*w*np.cos(w*t), 0, v0

def path_deriv_2(t):
    return -x0*w**2 * np.sin(w*t), 0, 0

The following functions are used for Newton's method when solving for the retarded time.

In [87]:
def rooter(tr, t, x, y, z):
    r = np.array([x,y,z])
    w = np.array(path(tr))
    return tr-t+np.sqrt(np.dot(r-w, r-w))

def rooter_slope(tr, t, x, y, z):
    r = np.array([x,y,z])
    w = np.array(path(tr))
    w_v = np.array(path_deriv(tr))

    numerator = np.dot(r-w, w_v)
    denominator = np.sqrt(np.dot(r-w, r-w))
    
    return 1 - numerator/denominator
    
def newtons_method(tr, t, x, y, z, N):
    for i in range(N):
        tr = tr - rooter(tr, t, x, y, z)/rooter_slope(tr, t, x, y, z)
    return tr

These methods are used to compute the electric field at a point $\vec{r}=(x,y,z)$ at a given time $t$ (note that the time $t$ specifies the location of the moving particle). Note that the retarded time depends both on $(x,y,z)$ and the time $t$ and needs to be taken into account here.

In [204]:
def elec_field(x, y, z, t):
    
    tr = newtons_method(t, t, x, y, z, 100)
    
    a = path_deriv_2(tr)
    
    script_r_vec = (np.array([x,y,z])-np.array(path(tr)))
    script_r = np.linalg.norm(script_r_vec)
    script_r_hat = (1/script_r)*script_r_vec
    
    Beta = np.array(path_deriv(tr))
    alpha = script_r_hat-Beta
    Beta_square = np.linalg.norm(script_r_vec)**2
    
    return script_r/(np.dot(script_r_vec, alpha))**3 * ((1-Beta_square)*alpha+
                                                       np.cross(script_r_vec, np.cross(alpha, a)))

Used to compute at each grid point.

In [209]:
def get_elec_field(X_d, Z_d, t):
    Rx = np.zeros(shape=X_d.shape)
    Ry = np.zeros(shape=X_d.shape)
    Rz = np.zeros(shape=X_d.shape)
    for i in range(len(X_d)):
        for j in range(len(X_d[i])):
            [field_x, field_y, field_z] = elec_field(X_d[i,j], 0, Z_d[i,j], t)
            Rx[i, j] = field_x
            Ry[i, j] = field_y
            Rz[i, j] = field_z
    return Rx, Ry, Rz

Specifies the array of times used.

In [210]:
x=1
y=1
z=1
t = 5
ttr = np.linspace(1, 10, 100)

Creates a pandas DataFrame which is used to store all the components of the fields at any given time. This is used because it makes handling animation easier, and provides an efficient way for storing data.

In [217]:
df = pd.DataFrame({'Time': ttr})
df['Position_Current'] = df['Time'].apply(lambda t: path(t))

X_d, Z_d = np.meshgrid(np.arange(-0.5, 0.5, .1), np.arange(0, 1 , .1))
df['Field'] = df['Time'].apply(lambda t: get_elec_field(X_d, Z_d, t))

df['Field_x'] = df['Field'].apply(lambda x: x[0])
df['Field_y'] = df['Field'].apply(lambda x: x[1])
df['Field_z'] = df['Field'].apply(lambda x: x[2])


mag = (df['Field_x']**2+df['Field_y']**2+df['Field_z']**2)**(1/2)
df['Field_x'] = df['Field_x']/mag
df['Field_y'] = df['Field_y']/mag
df['Field_z'] = df['Field_z']/mag
df['Mag'] = mag

Compute another column in the DataFrame which corresponds to the electric field magnitudes.

In [225]:
X_m, Z_m = np.meshgrid(np.arange(-0.5, 0.5, .05), np.arange(0, 1 , .05))
df['Field_mag'] = df['Time'].apply(lambda t: get_elec_field(X_m, Z_m, t))

df['Field_mag'] =  df['Field_mag'].apply(lambda x: (x[0]**2+x[1]**2+x[2]**2)**(1/2))

Here is a visualization of what the DataFrame looks like

In [114]:
df.head()

,Time,Position_Current,Field_x,Field_y,Field_z,Field_mag
0,1.000000,"(0.08414709848078966, 0, 0.1)","[[-0.985661428825882, -0.9793279348742167, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.16873514075295826, -0.2022790052748969, -...","[[2.8258099375740544, 2.922178121908998, 3.023..."
1,1.090909,"(0.08870469892594562, 0, 0.10909090909090909)","[[-0.983260610530334, -0.9759794463755447, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.18220475235161918, -0.2178626178408896, -...","[[2.7700404361907323, 2.863301077089307, 2.961..."
2,1.181818,"(0.09252970743422062, 0, 0.1181818181818182)","[[-0.9806837034457859, -0.9723986410491542, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.1956002908889905, -0.23332570129704563, -...","[[2.7216665116327032, 2.8122100769751865, 2.90..."
3,1.272727,"(0.09559053413015599, 0, 0.1272727272727273)","[[-0.9779212787208362, -0.9685705879024783, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.20897361705489192, -0.24873885150946484, ...","[[2.680371624832465, 2.7685607357727626, 2.861..."
4,1.363636,"(0.09786190034210546, 0, 0.13636363636363638)","[[-0.9749613122595449, -0.9644764940490125, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.22237454799762077, -0.26416868176777714, ...","[[2.6458392017380805, 2.732010177410661, 2.822..."


In [53]:
wx, wy, wz = path(ttr)

In [232]:
def animate(i):
    ax.clear()
    ax.pcolor(X_m, Z_m, df['Field_mag'][i], cmap=cmap, norm=colors.LogNorm(vmin=0.01, vmax=1000))
    ax.quiver(X_d, Z_d, df['Field_x'][i], df['Field_z'][i], units='width')
    ax.scatter(path(ttr[i])[0], path(ttr[i])[2], color='k')
    
    ax.set_ylim(0, 0.9)

In [233]:
fig, ax = plt.subplots(1,1, figsize=(6,6))

ani = animation.FuncAnimation(fig, animate, len(ttr), interval=1e2, blit=False)
#ani.save('eb_field.mp4', fps=30, dpi=300)
plt.close()

In [234]:
HTML(ani.to_html5_video())